In [71]:
import pandas as pd
from os import path
import pandas_profiling
pd.options.display.max_columns = 999
pd.options.display.max_rows = 1000
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import reverse_geocoder as rg
import numpy as np
from catboost import CatBoostClassifier
#from xgboost import XGBClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import seaborn as sns
import category_encoders as ce
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn.externals.six import StringIO  
import pydot
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [18]:
def read_data():
    if path.exists('./train_withextra.csv') and path.exists('./test_withextra.csv'):
        print('here')
        train_df = pd.read_csv('./train_withextra.csv',header=0)
        test_df = pd.read_csv('./test_withextra.csv',header=0)
    else:
        train_df = pd.read_csv('./Train.csv',header=0)
        test_df = pd.read_csv('./Test.csv',header=0)
        train_df = modiefied_train_name(train_df)
        train_df =data_preprocessing(train_df)
        test_df =data_preprocessing(test_df)
        train_df.to_csv('./train_withextra.csv')
        test_df.to_csv('./test_withextra.csv')
    #train_report = pandas_profiling.ProfileReport(train_df)
    #train_report.to_file(output_file = './trainreport.html')
    #test_report = pandas_profiling.ProfileReport(test_df)
    #test_report.to_file(output_file = './testreport.html')
    return train_df,test_df

In [19]:
def one_hot_enco(train,test):
    train['dataset_type'] = 'train'
    test['dataset_type'] = 'test'
    all_data_df = pd.concat([train, test])
    the_one_hot_encodings = pd.get_dummies(all_data_df['train_name'], prefix = 'train_name_')
    all_data_df = pd.concat( [all_data_df, the_one_hot_encodings ] , axis = 1)
    train_df = all_data_df[all_data_df['dataset_type'] == "train"]
    test_df = all_data_df[all_data_df['dataset_type'] == "test"]
    train_df.drop(['dataset_type'],axis=1,inplace=True)
    test_df.drop(['dataset_type'],axis=1,inplace=True)
    return train_df,test_df

In [20]:
def gbc_classifier(train_X,train_y,test_X,test_y): 
    GBC = GradientBoostingClassifier()
    gb_param_grid = {'loss' : ["deviance"],
                  'n_estimators' : [100,200,300],
                  'learning_rate': [0.1, 0.05, 0.01],
                  'max_depth': [4, 8],
                  'min_samples_leaf': [100,150],
                  'max_features': [0.3, 0.1] }
    gsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, cv=5, scoring="accuracy", n_jobs= 4, verbose = 1)
    gsGBC.fit(train_X,train_y)
    GBC_best = gsGBC.best_estimator_
    gsGBC.best_score_
    #print("Best Hyper Parameters:",model1.best_params_)
    print(classification_report(train_y,gsGBC.predict(train_X)))
    print(classification_report(test_y,gsGBC.predict(test_X)))
    return gsGBC

In [21]:
def decision_tree_classifier(train_X,train_y,test_X,test_y):
    #making the instance
    model= DecisionTreeClassifier(random_state=4)
    #Hyper Parameters Set
    params = {'max_features': [2,3,4,5,10,12,15,16,18,20,25,30,40,80,100,120,150,170,200],
              'min_samples_split': [4,5,6,7,8,9,10,11,12,13,14,15], 
              'min_samples_leaf':[5,6,7,8,9,10,11],
              'random_state':[47]}
    #Making models with hyper parameters sets
    model1 = GridSearchCV(model, param_grid=params, n_jobs=-1)
    #Learning
    model1.fit(train_X,train_y)
    #The best hyper parameters set
    print("Best Hyper Parameters:",model1.best_params_)
    print(classification_report(train_y,model1.predict(train_X)))
    print(classification_report(test_y,model1.predict(test_X)))
    return model1
    #submission = pd.DataFrame({'id_code':test_original['id_code'],'target':le.inverse_transform(pre)})
    #print("Accuracy:",metrics.accuracy_score(prediction,test_y))

In [22]:
def ranom_forest_classifier(train_X,train_y,test_X,test_y):
    from sklearn.model_selection import GridSearchCV
    from sklearn.ensemble import RandomForestClassifier
    #making the instance
    model=RandomForestClassifier()
    #hyper parameters set
    params = {'criterion':['gini','entropy'],
              'n_estimators':[10,15,20,25,30],
              'min_samples_leaf':[3,4,5,6,7,8,9],
              'min_samples_split':[3,4,5,6,7,8,9], 
              'random_state':[123],
              'n_jobs':[-1]}
    #Making models with hyper parameters sets
    model1 = GridSearchCV(model, param_grid=params, n_jobs=-1)
    #learning
    model1.fit(train_X,train_y)
    #The best hyper parameters set
    print("Best Hyper Parameters:\n",model1.best_params_)
    #Prediction
    prediction=model1.predict(test_X)
    #importing the metrics module
    print("Best Hyper Parameters:",model1.best_params_)
    print(classification_report(train_y,model1.predict(train_X)))
    print(classification_report(test_y,model1.predict(test_X)))
    return model1

In [23]:
def svm(train_X,train_y,test_X,test_y):
    from sklearn.model_selection import GridSearchCV
    from sklearn import svm
    #making the instance
    model=svm.SVC()
    #Hyper Parameters Set
    params = {'C': [6,7,8,9,10,11,12], 
              'kernel': ['linear','rbf']}
    #Making models with hyper parameters sets
    model1 = GridSearchCV(model, param_grid=params, n_jobs=-1,verbose = 4)
    #Learning
    model1.fit(train_X,train_y)
    #The best hyper parameters set
    print("Best Hyper Parameters:\n",model1.best_params_)
    #Prediction
    prediction=model1.predict(test_X)
    #importing the metrics module
    from sklearn import metrics
    #evaluation(Accuracy)
    print(classification_report(train_y,model1.predict(train_X)))
    print(classification_report(test_y,model1.predict(test_X)))
    return model1

In [24]:
def knn_classifier(train_X,train_y,test_X,test_y):
    from sklearn.model_selection import GridSearchCV
    from sklearn.neighbors import KNeighborsClassifier
    #making the instance
    model = KNeighborsClassifier(n_jobs=-1)
    #Hyper Parameters Set
    params = {'n_neighbors':[5,6,7,8,9,10],
              'leaf_size':[3,4,5],
              'weights':['uniform', 'distance'],
              'algorithm':['auto', 'ball_tree','kd_tree','brute'],
              'n_jobs':[-1]}
    #Making models with hyper parameters sets
    model1 = GridSearchCV(model, param_grid=params, n_jobs=1)
    #Learning
    model1.fit(train_X,train_y)
    #The best hyper parameters set
    print("Best Hyper Parameters:\n",model1.best_params_)
    #Prediction
    prediction=model1.predict(test_X)
    #importing the metrics module
    from sklearn import metrics
    #evaluation(Accuracy)
    print(classification_report(train_y,model1.predict(train_X)))
    print(classification_report(test_y,model1.predict(test_X)))
    return model1

In [63]:
def xgb_classifier(train_X,train_y,test_X,test_y):
    from sklearn.model_selection import RandomizedSearchCV
    params = {
            'min_child_weight': [1,2,3, 5, 6,7,8,9,10],
            'gamma': [0.1,0.3, 0.4,0.5,0.6,1, 1.5, 2,4, 5],
            'subsample': [0.1,0.2,0.4,0.6, 0.7,0.8, 1.0],
            'colsample_bytree': [0.1,0.3,0.6, 0.8, 1.0],
            'max_depth': [3, 4, 5,7,9]
            }
    xgb = XGBClassifier(learning_rate=0.01, n_estimators=1000, objective='multi:softmax',
                        silent=True, nthread=1,num_class = 3,num_boost_round=999,early_stopping_rounds=50)
    folds = 10
    param_comb = 15

    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

    random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, n_jobs=4, cv=skf.split(train_X,train_y), verbose=3, random_state=1001 )

    random_search.fit(train_X, train_y)
    print(random_search.cv_results_)
    print('\n Best estimator:')
    print(random_search.best_estimator_)
    #print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
    #print(random_search.best_score_ * 2 - 1)
    print('\n Best hyperparameters:')
    print(random_search.best_params_)
    print(classification_report(train_y,random_search.predict(train_X)))
    print(classification_report(test_y,random_search.predict(test_X)))
    return random_search

In [38]:
def convert24(str1): 
      
    # Checking if last two elements of time 
    # is AM and first two elements are 12 
    if str1[-2:] == "AM" and str1[:2] == "12":
        #print(str1[2:-2])
        return float('0' + '.'+ str1[3:5]) 
          
    # remove the AM     
    elif str1[-2:] == "AM": 
        return float(str1[:2] + '.' + str1[3:5]) 
      
    # Checking if last two elements of time 
    # is PM and first two elements are 12    
    elif str1[-2:] == "PM" and str1[:2] == "12": 
        return float(str1[:2] + '.' + str1[3:5])  
          
    else: 
          
        # add 12 to hours and remove PM 
        temp = str(int(str1[:2]) + 12) + str1[2:8]
        return float(temp[:2] + '.' + temp[3:5])
        #return str(temp[:2] + '.'+ temp[3:5])

def modiefied_train_name(df):
    df.loc[df['train_name'] == 'UNDEFINED', 'train_name'] = 'ICZVZS'
    return df

def convertdecimal(str1):
    return float(str1[:2] + '.' + str1[3:4])

def columns_tobe_ignored(df,cols):
    df.drop(columns =cols, inplace = True)
    return df

def label_encoding(train,test):
    #  label encoding to all the columns
    le=LabelEncoder()
    for col in test.columns.values:
        if (test[col].dtypes=='object') or (test[col].dtypes == 'bool'):
            data=train[col].append(test[col])
            le.fit(data.values)
            train[col]=le.transform(train[col])
            test[col]=le.transform(test[col])            
    return train,test

def label_encoding_target(train):
    le=LabelEncoder()
    train['target']=le.fit_transform(train['target'])
    return le,train

def lat_long_2_name(lat,long):
    if not (np.isnan(lat) or np.isnan(long)):
        coordinates = (lat,long)
        #print(coordinates)
        rt = rg.search(coordinates)
        #print(rt)
        return rt[0].get('name')
    else:
        return ''
    
def trains_on_weekends(df):
    first = df.loc[df['is_weekend'] == True]
    weekend_trains = first.train_name.unique()
    second = df.loc[df['is_weekend'] == False]
    weekday_trains = second.train_name.unique()
    li = [x for x in weekend_trains if x not in weekday_trains]
    inex = df['train_name'].isin(li)
    df['is_weekend_train'] = inex
    return df

def non_intersection_target(df):
    low = df.loc[df['target'] == 'low']
    low_trains = low.train_name.unique()
    medium = df.loc[df['target'] == 'medium']
    medium_trains = medium.train_name.unique()
    high = df.loc[df['target'] == 'high']
    high_trains = high.train_name.unique()
    print('intersection of all 3 trains')
    #print(set(low_trains) & set(medium_trains) & set(high_trains))
    print('\n')
    print('exclusive low')
    #print(set(low_trains) - (set(medium_trains) | set(high_trains)))
    print('\n')
    print('exclusive high')
    #print(set(high_trains) - (set(low_trains) | set(medium_trains)))
    print('\n')
    print('exclusive medium')
    #print(set(medium_trains) - (set(low_trains) | set(high_trains)))
#     lisource = []
#     lidestination = []
#     for index, row in df.iterrows():
#         coordinates = (row['latitude_source'],row['longitude_source'])
#         rt = rg.search(coordinates)
#         lisource.append(rt[0].get('name'))
        
#         coordinates = (row['latitude_destination'],row['longitude_destination'])
#         rt = rg.search(coordinates)
#         lidestination.append(rt[0].get('name'))
#     df['source_address'] = lisource
#     df['destination_address'] = lidestination
#    return df
        
#dataset3.head() 
def exploratory_analyis(df):
    sns.FacetGrid(df,hue = 'target', size = 5).map(sns.distplot,'source_num').add_legend()

def hash_encoding(train,test):
    categorical_columns = ['current_day', 'is_weekend', 'destination_address',
       'source_address', 'is_weekend_train']
    #for col in categorical_columns:
        #if (test[col].dtypes=='object') or (test[col].dtypes == 'bool'):
        #    categorical_columns.append(col)
    he = ce.HashingEncoder(cols=categorical_columns, n_components=len(categorical_columns))
    train_he = he.fit_transform(train)
    test_he = he.transform(test)
    return he,train_he,test_he

In [53]:
def data_preprocessing(df):
    df['source_num'] = df.source_name.str.partition("station$",True)[2]
    df['destination_num'] = df.destination_name.str.partition("station$",True)[2]
    df['current_decimal_time'] = df.apply(lambda x: convert24(x['current_time']), axis=1)
    df['source_address'] = df.apply(lambda x: lat_long_2_name(x['latitude_source'],x['longitude_source']), axis=1)
    df['destination_address'] = df.apply(lambda x: lat_long_2_name(x['latitude_destination'],x['longitude_destination']), axis=1)
    return df

In [54]:
def datascaling(train_frame,test_frame):
    from sklearn.preprocessing import StandardScaler,MinMaxScaler
    #longitude_source,latitude_source,mean_halt_times_source,longitude_destination,latitude_destination,mean_halt_times_destination
#current_week,source_num,destination_num,current_decimal_time,
    columns = ['current_week', 'destination_num',
       'latitude_destination', 'latitude_source',
       'longitude_destination', 'longitude_source',
       'mean_halt_times_destination', 'mean_halt_times_source','source_num','current_decimal_time']
    std_scale = MinMaxScaler().fit(train_frame[columns])
    train_frame[columns] = std_scale.transform(train_frame[columns])
    test_frame[columns]  = std_scale.transform(test_frame[columns])
    return train_frame,test_frame

In [56]:
train_frame,test_frame = read_data()
train_original = train_frame.copy()
test_original = test_frame.copy()
train_frame = trains_on_weekends(train_frame)
test_frame = trains_on_weekends(test_frame)
y = train_frame['target']
train_frame = train_frame.drop(['target'],axis=1)
he,train_he_frame,test_he_frame = hash_encoding(train_frame,test_frame)
#print(train_he_frame.tail(50))
train_he_frame,test_he_frame = one_hot_enco(train_he_frame,test_he_frame)
#print(train_he_frame.columns)
#print(test_he_frame.columns)

train_frame = columns_tobe_ignored(train_he_frame,['id_code','current_date','current_time','source_name','destination_name',\
                                                'country_code_source','current_year','country_code_destination','train_name'])

test_frame = columns_tobe_ignored(test_he_frame,['id_code','current_date','current_time','source_name','destination_name',\
                                                'country_code_source','current_year','country_code_destination','train_name'])



#print('Fore test\n')
#test_frame = data_preprocessing(test_frame)
#print(test_frame.columns)

#print(train_frame.info())
#longitude_source,latitude_source,mean_halt_times_source,longitude_destination,latitude_destination,mean_halt_times_destination
#current_week,source_num,destination_num,current_decimal_time,

here


In [57]:
train_frame.fillna(train_frame.mean(), inplace=True)
#le,train_frame = label_encoding_target(train_frame)
#y = label_binarize(y, classes=[0, 1, 2])
train_frame,test_frame = datascaling(train_frame,test_frame)
#train_he_frame,test_he_frame = hash_encoding(train_frame,test_frame)
#y = train_frame['target']
#train_X = train_frame.drop(['target'],axis=1)
columns_to_scale  = ['gre', 'gpa']

#test_X = test_frame.drop(['Unnamed: 0'],axis=1)
#train_he_frame,test_he_frame = train_X,test_X
#hashenc,train_he_frame,test_he_frame = hash_encoding(train_X,test_X)
#print(hashenc.get_feature_names())
#train_frame = train_frame[train_frame.longitude_source > 0]

In [58]:
train_frame.head(10)
test_frame.tail(10)

,col_0,col_1,col_2,col_3,col_4,longitude_source,latitude_source,mean_halt_times_source,longitude_destination,latitude_destination,mean_halt_times_destination,current_week,source_num,destination_num,current_decimal_time,train_name__EURRZAS,train_name__ICEZZ,train_name__ICRAU,train_name__ICRWZ,train_name__ICRXS,train_name__ICRXU,train_name__ICRYR,train_name__ICRYUA,train_name__ICRYUX,train_name__ICRYVY,train_name__ICRYVZ,train_name__ICRYWA,train_name__ICRYXZ,train_name__ICRYYA,train_name__ICRYYS,train_name__ICRYYW,train_name__ICRYYX,train_name__ICRYZR,train_name__ICRYZU,train_name__ICRYZV,train_name__ICRZZ,train_name__ICSAT,train_name__ICSXR,train_name__ICSXT,train_name__ICSXV,train_name__ICSXY,train_name__ICSYR,train_name__ICSYY,train_name__ICSZU,train_name__ICSZV,train_name__ICTAR,train_name__ICTAS,train_name__ICTAT,train_name__ICTAU,train_name__ICTURVY,train_name__ICTURZA,train_name__ICTUSAY,train_name__ICTWY,train_name__ICTXR,train_name__ICTXT,train_name__ICTXU,train_name__ICTXV,train_name__ICTXW,train_name__ICTXX,train_name__ICTYR,train_name__ICTYS,train_name__ICTYX,train_name__ICTYY,train_name__ICTZA,train_name__ICTZR,train_name__ICTZS,train_name__ICTZU,train_name__ICTZV,train_name__ICTZW,train_name__ICTZX,train_name__ICTZY,train_name__ICUZZ,train_name__ICVAR,train_name__ICVAS,train_name__ICVAT,train_name__ICVAU,train_name__ICVWA,train_name__ICVWV,train_name__ICVWW,train_name__ICVWX,train_name__ICVWY,train_name__ICVXA,train_name__ICVXR,train_name__ICVXS,train_name__ICVXSR,train_name__ICVXSV,train_name__ICVXSX,train_name__ICVXSY,train_name__ICVXT,train_name__ICVXU,train_name__ICVXUR,train_name__ICVXUT,train_name__ICVXV,train_name__ICVXW,train_name__ICVXX,train_name__ICVXY,train_name__ICVYA,train_name__ICVYR,train_name__ICVYS,train_name__ICVYY,train_name__ICVYZ,train_name__ICVZA,train_name__ICVZR,train_name__ICVZS,train_name__ICVZT,train_name__ICVZU,train_name__ICVZV,train_name__ICVZW,train_name__ICVZX,train_name__ICVZY,train_name__ICVZZ,train_name__ICWAAV,train_name__ICWAR,train_name__ICWAS,train_name__ICWAT,train_name__ICWAWY,train_name__ICWAXR,train_name__ICWUYR,train_name__ICWVAT,train_name__ICWVAU,train_name__ICWVWY,train_name__ICWVWZ,train_name__ICWVXA,train_name__ICWVXS,train_name__ICWVXU,train_name__ICWVXV,train_name__ICWVXX,train_name__ICWVYA,train_name__ICWVYR,train_name__ICWVYS,train_name__ICWVZR,train_name__ICWVZU,train_name__ICWVZV,train_name__ICWVZX,train_name__ICWWA,train_name__ICWWXX,train_name__ICWWY,train_name__ICWWYA,train_name__ICWWZ,train_name__ICWXA,train_name__ICWXR,train_name__ICWXT,train_name__ICWXU,train_name__ICWXW,train_name__ICWXWA,train_name__ICWXWY,train_name__ICWXXA,train_name__ICWXXT,train_name__ICWXXV,train_name__ICWXXX,train_name__ICWXYU,train_name__ICWXYW,train_name__ICWXZ,train_name__ICWXZR,train_name__ICWXZT,train_name__ICWXZY,train_name__ICWYA,train_name__ICWYAR,train_name__ICWYAT,train_name__ICWYR,train_name__ICWYS,train_name__ICWYXA,train_name__ICWYXR,train_name__ICWYXT,train_name__ICWYXU,train_name__ICWYYA,train_name__ICWYZA,train_name__ICWYZU,train_name__ICWZA,train_name__ICWZAS,train_name__ICWZAT,train_name__ICWZAU,train_name__ICWZT,train_name__ICWZU,train_name__ICWZV,train_name__ICWZW,train_name__ICWZWA,train_name__ICWZWZ,train_name__ICWZXR,train_name__ICWZXS,train_name__ICWZXW,train_name__ICWZXX,train_name__ICWZY,train_name__ICWZYS,train_name__ICWZYY,train_name__ICWZZ,train_name__ICWZZT,train_name__ICWZZV,train_name__ICXAAT,train_name__ICXAAU,train_name__ICXAXS,train_name__ICXRXV,train_name__ICXRZS,train_name__ICXRZX,train_name__ICXSAV,train_name__ICXSAW,train_name__ICXSXA,train_name__ICXSXS,train_name__ICXSXT,train_name__ICXSXV,train_name__ICXSXW,train_name__ICXSXY,train_name__ICXSYT,train_name__ICXSZW,train_name__ICXSZY,train_name__ICXSZZ,train_name__ICXTAR,train_name__ICXTAS,train_name__ICXTAT,train_name__ICXTAU,train_name__ICXTAX,train_name__ICXTXA,train_name__ICXTXS,train_name__ICXTXT,train_name__ICXTXU,train_name__ICXTXV,train_name__ICXTYA,train_name__ICXTYR,train_name__ICX

In [72]:
#print(train_he_frame.head(30))
#print(y)
X_train, X_vali, y_train, y_vali = train_test_split(train_frame, y, test_size=0.20)
#modelde = decision_tree_classifier(X_train, y_train,X_vali, y_vali)
#pre = model.predict(test_he_frame)
#print('\n')
#modelra = ranom_forest_classifier(X_train, y_train,X_vali, y_vali)
#modelknn = knn_classifier(X_train, y_train,X_vali, y_vali)
modelxgb =XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1.0,
              early_stopping_rounds=100, gamma=0.4, learning_rate=0.01,
              max_delta_step=0, max_depth=9, min_child_weight=3, missing=None,
              n_estimators=1000, n_jobs=1, nthread=1, num_boost_round=999,
              num_class=3, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=True, subsample=0.8, verbosity=1)


#,X_vali, y_vali)
#modelde = DecisionTreeClassifier(random_state=4,min_samples_split = 13, max_features = 15, min_samples_leaf = 5)
modelxgb.fit(X_train, y_train,verbose=3)

#dot_data = StringIO() 
#tree.export_graphviz(modelde, out_file=dot_data)
#tree.export_graphviz(modelde, out_file=dot_data, feature_names=hashenc.get_feature_names())
#graph = pydot.graph_from_dot_data(dot_data.getvalue()) 

#graph[0].write_pdf("./hitachi.pdf")  # must access graph's first element
#modelxgb = xgb_classifier(X_train, y_train,X_vali, y_vali)
#modelgbc = gbc_classifier(X_train, y_train,X_vali, y_vali)
#submission = pd.DataFrame({'id_code':test_original['id_code'],'target':modelde.predict(test_he_frame)})
#submission.to_csv('./submission.csv')


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1.0,
              early_stopping_rounds=100, gamma=0.4, learning_rate=0.01,
              max_delta_step=0, max_depth=9, min_child_weight=3, missing=None,
              n_estimators=1000, n_jobs=1, nthread=1, num_boost_round=999,
              num_class=3, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=True, subsample=0.8, verbosity=1)

In [67]:
print(classification_report(y_train,modelxgb.predict(X_train)))
print(classification_report(y_vali,modelxgb.predict(X_vali)))

              precision    recall  f1-score   support

        high       0.96      0.98      0.97       302
         low       0.98      0.98      0.98       442
      medium       0.99      0.97      0.98       283

    accuracy                           0.98      1027
   macro avg       0.98      0.98      0.98      1027
weighted avg       0.98      0.98      0.98      1027

              precision    recall  f1-score   support

        high       0.68      0.63      0.66        90
         low       0.59      0.74      0.66        90
      medium       0.53      0.40      0.46        77

    accuracy                           0.60       257
   macro avg       0.60      0.59      0.59       257
weighted avg       0.60      0.60      0.60       257



In [70]:
submission = pd.DataFrame({'id_code':test_original['id_code'],'target':modelxgb.predict(test_frame)})
submission.to_csv('./submission.csv',index = False)